In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt

def load_quantum_state_data(image_folder_path, target_size=(128, 128)):
    """
    Load and preprocess quantum state images from the specified folder.
    Returns images and their corresponding principal quantum numbers (n).
    """
    images = []
    labels = []
    skipped_files = []
    
    for filename in os.listdir(image_folder_path):
        if filename.endswith('.png'):
            try:
                # Extract n from filename (format: n_l_m.png)
                n = int(filename.split('_')[0])
                
                # Load and preprocess image
                image_path = os.path.join(image_folder_path, filename)
                image = Image.open(image_path).convert('RGB')
                image = image.resize(target_size)
                images.append(np.array(image))
                labels.append(n)
                
            except (ValueError, IndexError):
                skipped_files.append(filename)
    
    print(f"Loaded {len(images)} images and {len(labels)} labels.")
    print(f"Skipped {len(skipped_files)} files due to naming convention issues.")
    print(f"Unique n values found: {np.unique(labels)}")
    
    return np.array(images), np.array(labels)

def create_cnn_model(input_shape, num_classes):
    """
    Create a simple CNN model for quantum state classification.
    """
    model = models.Sequential([
        # First Convolutional Block
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        
        # Second Convolutional Block
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Third Convolutional Block
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Flatten and Dense Layers
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

def train_model():
    # Load and preprocess data
    train_dir = 'training-data'  # Directory containing training images
    X_train, y_train = load_quantum_state_data(train_dir)
    
    # Data preprocessing
    X_train = X_train.astype('float32') / 255.0
    max_n = np.max(y_train)
    y_train = tf.keras.utils.to_categorical(y_train - 1, max_n)
    
    # Create and compile model
    model = create_cnn_model((128, 128, 3), max_n)
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy']
    )
    
    # Display model summary
    model.summary()
    
    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=200,
        batch_size=16,
        validation_split=0.2
    )
    
    return model, history

def predict_quantum_state(model, image_path):
    """
    Predict the principal quantum number of a given quantum state image.
    """
    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    image = image.resize((128, 128))
    image = np.array(image)
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)
    
    # Make prediction
    prediction = model.predict(image)
    predicted_n = np.argmax(prediction) + 1
    
    return predicted_n

def main():
    # Train the model
    print("Training model...")
    model, history = train_model()
    
    # Test the model on a new image
    test_dir = 'classification-data'
    X_test, y_test = load_quantum_state_data(test_dir)
    
    # Preprocess test data
    X_test = X_test.astype('float32') / 255.0
    
    # Make predictions
    predictions = model.predict(X_test)
    predicted_n = np.argmax(predictions, axis=1) + 1
    
    print(f"Predicted principal quantum numbers (n): {predicted_n}")
    
if __name__ == "__main__":
    main()

Training model...
Loaded 84 images and 84 labels.
Skipped 0 files due to naming convention issues.
Unique n values found: [1 2 3 4 5 6 7]


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       802,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 859,655 (3.28 MB)

 Trainable params: 859,655 (3.28 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - categorical_accuracy: 0.2441 - loss: 1.7678 - val_categorical_accuracy: 0.1765 - val_loss: 3.0438
Epoch 2/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - categorical_accuracy: 0.3555 - loss: 1.4367 - val_categorical_accuracy: 0.1765 - val_loss: 2.2392
Epoch 3/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - categorical_accuracy: 0.4054 - loss: 1.3840 - val_categorical_accuracy: 0.1765 - val_loss: 2.2095
Epoch 4/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - categorical_accuracy: 0.3483 - loss: 1.2822 - val_categorical_accuracy: 0.1765 - val_loss: 1.9458
Epoch 5/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - categorical_accuracy: 0.3939 - loss: 1.1556 - val_categorical_accuracy: 0.1765 - val_loss: 2.0174
Epoch 6/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - categorical_accuracy: 0.6215 - loss: 0.9369 - val_categorical_accuracy: 0.2353 - val_loss: 2.1644
Epoch 7/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - categorical_accuracy: 0.5992 - loss: 0.8467 - va